In [0]:
from pyspark.sql.functions import current_timestamp

# Load the source table
player_activity_df = spark.table("dev_catalog.silver.player_activity")

# Select distinct activity types and add insert_date and update_date
dim_activity_type_df = player_activity_df.select("Activity_type").distinct() \
    .withColumn("insert_date", current_timestamp()) \
    .withColumn("update_date", current_timestamp())

# Create a temporary view for the dimension table
dim_activity_type_df.createOrReplaceTempView("temp_dim_activity_type")

# Merge the data into the target table
merge = ("""
    MERGE INTO dev_catalog.gold.dim_activity_type AS target
    USING temp_dim_activity_type AS source
    ON target.Activity_type = source.Activity_type
    WHEN MATCHED THEN
        UPDATE SET target.update_date = source.update_date
    WHEN NOT MATCHED THEN
        INSERT (Activity_type, insert_date, update_date)
        VALUES (source.Activity_type, source.insert_date, source.update_date)
""")

spark.sql(merge)

# Display the dimension table
#display(spark.table("dev_catalog.gold.dim_activity_type"))